In [11]:
knitr::opts_chunk$set(echo = F)
library(dplyr)
library(stats)
library(metafor)
library(ggplot2)
library(shinyWidgets)
library(tidyr)
library(shiny)


# Model

This notebook simulates a two-sample MR study by generating individual-level data for two non-overlapping samples and then calculating the requiorange summary statistics. The models below are based on those from Bowden et al. (2015).

The simulation includes an unobserved confounder $C_i$, an exposure $X_i$, and an outcome $Y_i$ for each individual $i$. The genotype $G_{ij}$ is a vector of genetic variants for individual $i$ and variant $j$. $G_{ij}\in \{0,1,2\}$ is generated from a binomial distribution with a minor allele frequency of 0.3.

**Confounder model**

$$
C_i = \sum_{j=1}^{J} \phi_j G_{ij} + \varepsilon_i^C
$$

where

-   $\phi_j$ is the strength of the correlated pleiotropy - Under InSIDE (no correlated pleiotropy), $\phi_j = 0$.
-   Under correlated pleiotropy, $\phi_j \sim \operatorname{Uniform}(0, 0.5)$.
-   $\varepsilon_i^C$ is a random error drawn from a standard normal distribution.

**Exposure model**

$$
X_i = \sum_{j=1}^{J} \gamma_j G_{ij} + C_i + \varepsilon_{i}^{X}
$$

where

-   $\gamma_j \sim \operatorname{Uniform}(0.5, 2\, \bar \gamma - 0.5)$ is the strength of the association between the genetic variant and the exposure.
-   $\bar \gamma$ is the average instrument strength.
-   $\varepsilon_{i}^{X}$ is a random error drawn from a standard normal distribution.

**Outcome model**

$$
Y_i = \beta X_i + \sum_{j=1}^{J} \pi_j \alpha_j G_{ij} + C_i + \varepsilon_{i}^{Y}
$$

where

-   $\beta$ is the true effect size of the exposure on the outcome.
-   $\pi_j \sim \operatorname{Bernoulli}(p_{\text{pleio}})$ determines if variant $j$ is a pleiotropic variant with probability $p_\text{pleio}$.
-   $\alpha_j$ is the strength of the pleiotropic effect of variant $j$ on the outcome.
    -   Under no pleiotropy (valid IVs), $\alpha_j = 0$.
    -   Under balanced pleiotropy, $\alpha_j \sim \operatorname{Uniform}(-0.2, 0.2)$.
    -   Under directional pleiotropy, $\alpha_j \sim \operatorname{Uniform}(0, 0.2)$.
-   $\varepsilon_{i}^{Y}$ is a random error drawn from a standard normal distribution.

**Generating individual level data**

To approximate the two-sample MR setting we generate two data sets of participant genotypes as follows:

-   Generate matrices $G_1$ and $G_2$.
-   Generate confounder vectors $C_1$ and $C_2$.
-   Generate exposure vectors $X_1$ and $X_2$.
-   Generate outcome vector $Y_2$.

**Estimating GWAS coefficients**

Finally we estimate GWAS coefficients for each of the samples. Let $G[:, j]$ denote the $j$th column of matrix G, then we get the SNP-trait associations as follows:

-   Sample 1: For each SNP $j$, estimate its SNP-exposure association as $\hat \gamma_j = \operatorname{lm}(X_1 \sim G_1[:, j])$.
-   Sample 2: For each SNP $j$, estimate its SNP-outcome association as $\hat \Gamma_j = \operatorname{lm}(Y_2 \sim G_2[:, j])$.

# Simulation code setup

In [3]:
# --- Simulation Parameters ---
J <- 20 # Number of SNPs to simulate
N <- 1000 # Individuals in exposure sample
MAF <- 0.3 # Minor Allele Frequency
MEAN_GAMMA <- 3 # Mean of gamma (SNP effects on exposure)
BETA <- 0.05 # True causal effect of X on Y
P_PLEIO <- 0.3 # Proportion of pleiotropic SNPs


In [4]:
# Function to estimate SNP coefficients using OLS
estimate_coefficients <- function(G, trait) {
  results_list <- list()

  for (j in 1:ncol(G)) {
    df_snp <- data.frame(snp = G[, j], trait = trait)
    
    # Use lm for linear regression
    model <- lm(trait ~ snp, data = df_snp)
    
    # Extract coefficients, standard errors, and p-values
    c <- coef(model)["snp"]
    std_err <- summary(model)$coefficients["snp", "Std. Error"]
    p_value <- summary(model)$coefficients["snp", "Pr(>|t|)"]
    
    # Store the results
    results_list[[j]] <- data.frame(
      snp_index = j - 1, 
      coef = c,
      std_err = std_err,
      p_value = p_value
    )
  }
  return(do.call(rbind, results_list))
}

In [5]:

# Function to generate individual-level data
generate_individual <- function(
  beta = BETA,
  J = J,
  N = N,
  maf = MAF,
  mean_gamma = MEAN_GAMMA,
  mean_alpha = 0,
  is_pleiotropy = FALSE,
  is_inside = TRUE,
  pleio_proportion = P_PLEIO,
  seed = NULL
) {
  if (!is.null(seed)) {
    set.seed(seed)
  }

  if (is_pleiotropy) {
    # Generate pleiotropic effects
    alphas <- runif(J, -0.2, 0.2) + mean_alpha
  } else {
    alphas <- rep(0, J)
  }

  if (is_inside) {
    phis <- rep(0, J)
  } else {
    phis <- runif(J, 0, 0.5)
  }

  pis <- rbinom(J, 1, pleio_proportion) # Pleiotropic SNPs
  gammas <- runif(J, 0.5, 2 * MEAN_GAMMA - 0.5) # SNP effects on exposure

  G1 <- matrix(rbinom(N * J, 2, maf), nrow = N, ncol = J)
  G2 <- matrix(rbinom(N * J, 2, maf), nrow = N, ncol = J)

  # Noise terms
  eps_scale <- 2
  epsilon_C <- rnorm(N, 0, eps_scale)
  epsilon_X <- rnorm(N, 0, eps_scale)
  epsilon_Y <- rnorm(N, 0, eps_scale)

  # Confounders
  C1 <- G1 %*% phis + epsilon_C
  C2 <- G2 %*% phis + epsilon_C

  # Exposure
  X1 <- G1 %*% gammas + C1 + epsilon_X
  X2 <- G2 %*% gammas + C2 + epsilon_X

  # Outcome only generated from sample 2
  Y <- beta * X2 + G2 %*% (alphas * pis) + C2 + epsilon_Y

  return(list(
    G1 = G1,
    G2 = G2,
    C1 = C1,
    C2 = C2,
    X1 = as.vector(X1), # Ensure these are vectors for lm
    X2 = as.vector(X2),
    Y = as.vector(Y)
  ))
}


In [6]:

# Function to run the full two-sample MR simulation
simulate_model <- function(
  beta = BETA,
  J = J,
  N = N,
  maf = MAF,
  mean_gamma = MEAN_GAMMA,
  mean_alpha = 0,
  is_pleiotropy = FALSE,
  is_inside = TRUE,
  pleio_proportion = P_PLEIO,
  seed = NULL
) {
  if (!is.null(seed)) {
    set.seed(seed)
  }

  data <- generate_individual(
    beta = beta,
    J = J,
    N = N,
    maf = maf,
    mean_gamma = mean_gamma,
    mean_alpha = mean_alpha,
    is_pleiotropy = is_pleiotropy,
    is_inside = is_inside,
    pleio_proportion = pleio_proportion,
    seed = seed
  )

  gamma_hat_df <- estimate_coefficients(data$G1, data$X1)
  Gamma_hat_df <- estimate_coefficients(data$G2, data$Y)

  gamma_hat <- gamma_hat_df$coef
  Gamma_hat <- Gamma_hat_df$coef
  gamma_se <- gamma_hat_df$std_err
  Gamma_se <- Gamma_hat_df$std_err
  gamma_p <- gamma_hat_df$p_value
  Gamma_p <- Gamma_hat_df$p_value

  df_sim <- data.frame(
    gamma_hat = gamma_hat,
    Gamma_hat = Gamma_hat,
    gamma_se = gamma_se,
    Gamma_se = Gamma_se,
    gamma_p = gamma_p,
    Gamma_p = Gamma_p,
    weight = 1 / (Gamma_se^2),
    wald_ratio = Gamma_hat / gamma_hat # Wald ratio estimate
  ) %>%
    arrange(gamma_hat)

  return(df_sim)
}

In [7]:

# Compute IVW estimate
compute_ivw <- function(df_reg) {
  # Perform weighted least squares regression
  # Note: R's lm() with 'weights' performs WLS directly.
  # The formula needs to be specified with the dependent variable first.
  wls_model <- lm(Gamma_hat ~ 0 + gamma_hat, data = df_reg, weights = weight)
  
  # For a model without an intercept (0 + gamma_hat), the coefficient is directly gamma_hat
  beta_ivw <- coef(wls_model)["gamma_hat"] 
  
  return(list(beta_ivw = beta_ivw, wls_model = wls_model))
}

In [8]:
# Compute Egger estimate
compute_egger <- function(df_reg) {
  # Perform weighted least squares regression with intercept
  wls_model <- lm(Gamma_hat ~ gamma_hat, data = df_reg, weights = weight)
  
  alpha <- coef(wls_model)["(Intercept)"]
  beta <- coef(wls_model)["gamma_hat"]
  
  return(list(alpha = alpha, beta = beta, wls_model = wls_model))
}


In [9]:
# Execute the simulation and estimate coefficients
data = simulate_model(
  beta = BETA,
  J = 100,
  N = 5000,
  maf = MAF,
  mean_gamma = MEAN_GAMMA,
  mean_alpha = 0,
  is_pleiotropy = FALSE,
  is_inside = TRUE,
  pleio_proportion = P_PLEIO,
  seed = NULL
) 
head(data)

,gamma_hat,Gamma_hat,gamma_se,Gamma_se,gamma_p,Gamma_p,weight,wald_ratio
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-0.5793982,0.018432311,0.4728446,0.06658769,0.22050265,0.78193547,225.5340,-0.03181285
2,-0.1312491,0.009400255,0.4707288,0.06677138,0.78039365,0.88804729,224.2948,-0.07162147
3,0.4700276,0.119736167,0.4697286,0.06532776,0.31705098,0.06688392,234.3174,0.25474284
4,0.4934235,0.020986754,0.4760883,0.06625276,0.30006023,0.75143290,227.8201,0.04253294
5,0.6084456,0.053812561,0.4708240,0.06609648,0.19631325,0.41559685,228.8987,0.08844268
6,0.8181563,0.114484780,0.4738926,0.06608352,0.08432662,0.08325950,228.9885,0.13993021


# Simulations

## IVW estimate

### No pleiotropy

In [12]:
### UI controls
sliderInput("beta_slider_np", "Beta:", 
min = 0.0, max = 0.5, value = 0.05, step = 0.01)

Shiny tags cannot be represented in plain text (need html)

In [13]:
sliderInput("J_slider_np", "J (SNPs):",
min = 10, max = 200, value = 50, step = 1)


Shiny tags cannot be represented in plain text (need html)

In [14]:
sliderInput("N_slider_np", "N (Sample Size):", 
min = 250, max = 10000, value = 5000, step = 250)


Shiny tags cannot be represented in plain text (need html)

In [15]:
actionButton("run_button_np", "Run Simulation (No Pleiotropy)") 


Shiny tags cannot be represented in plain text (need html)

In [17]:

## Plotting Logic
reactive_plot_data_np <- eventReactive(input$run_button_np, {
current_beta <- input$beta_slider_np 
current_J <- input$J_slider_np      
current_N <- input$N_slider_np     

df_sim <- simulate_model(
beta = current_beta,
is_pleiotropy = FALSE,
is_inside = TRUE,
J = current_J,
N = current_N,
# Explicitly set these for this scenario as they are not controlled by sliders here
mean_alpha = 0,
pleio_proportion = 0
)

ivw_result <- compute_ivw(df_sim)
beta_ivw <- ivw_result$beta_ivw

df_sim$ivw <- df_sim$gamma_hat * beta_ivw
df_sim$true <- df_sim$gamma_hat * current_beta

return(df_sim)
}, ignoreNULL = FALSE)

In [18]:

# Render the plot with specific width and height arguments directly in renderPlot()
renderPlot({
df_plot <- reactive_plot_data_np()

ggplot(df_plot, aes(x = gamma_hat, y = Gamma_hat)) +
geom_point(aes(size = weight), alpha = 0.6) +
geom_line(aes(y = ivw, color = "IVW Regression")) +
geom_line(aes(y = true, color = "True Effect"), linetype = "dashed") +
labs(
title = "Scatter plot of SNP Effects (No Pleiotropy)",
x = "SNP-exposure effect (gamma_hat)",
y = "SNP-outcome effect (Gamma_hat)"
) +
scale_color_manual(
name = "Line Type",
values = c("IVW Regression" = "orange", "True Effect" = "black")
) +
theme_minimal() +
theme(legend.position = "right",
      legend.text = element_text(size = 18),
      legend.title = element_text(size = 18, face = "bold"),
      axis.title = element_text(size = 18),
      plot.title = element_text(hjust = 0.5, face = "bold", size = 16) 
      )
}, width = 800, height = 600)



### Balanced pleiotropy
Assume all SNPs are balanced pleiotropic.


In [20]:
### UI controls
# Use shiny::sliderInput for sliders
sliderInput("beta_slider_bp", "Beta:",
            min = 0.0, max = 0.5, value = 0.05, step = 0.01)

sliderInput("J_slider_bp", "J (SNPs):",
            min = 10, max = 200, value = 50, step = 1)

sliderInput("N_slider_bp", "N (Sample Size):",
            min = 250, max = 10000, value = 5000, step = 250)

sliderInput("mean_alpha_slider_bp", "Mean alpha:",
            min = 0, max = 3, value = 0, step = 0.1)


Shiny tags cannot be represented in plain text (need html)

Shiny tags cannot be represented in plain text (need html)

Shiny tags cannot be represented in plain text (need html)

Shiny tags cannot be represented in plain text (need html)

In [21]:
# Use shiny::actionButton for the "Run Simulation" button
actionButton("run_button_bp", "Run Simulation (Balanced Pleiotropy)")

Shiny tags cannot be represented in plain text (need html)

In [22]:

## Plotting Logic
# This reactive expression will re-run when input$run_button_bp is pressed
reactive_plot_data_bp <- eventReactive(input$run_button_bp, {
  current_beta <- input$beta_slider_bp
  current_J <- input$J_slider_bp      
  current_N <- input$N_slider_bp      
  current_mean_alpha <- input$mean_alpha_slider_bp
  
  df_sim <- simulate_model(
    beta = current_beta,
    mean_alpha = current_mean_alpha,
    is_pleiotropy = TRUE,
    is_inside = TRUE,
    pleio_proportion = 1,
    J = current_J,
    N = current_N
  )
  
  ivw_result <- compute_ivw(df_sim)
  beta_ivw <- ivw_result$beta_ivw
  
  df_sim$ivw <- df_sim$gamma_hat * beta_ivw
  df_sim$true <- df_sim$gamma_hat * current_beta
  
  return(df_sim)
}, ignoreNULL = FALSE)

# Render the plot with specific width and height arguments directly in renderPlot()
renderPlot({
  df_plot <- reactive_plot_data_bp()
  
  ggplot(df_plot, aes(x = gamma_hat, y = Gamma_hat)) +
    geom_point(aes(size = weight), alpha = 0.6) +
    geom_line(aes(y = ivw, color = "IVW Regression")) +
    geom_line(aes(y = true, color = "True Effect"), linetype = "dashed") +
    labs(
      title = "Scatter plot of SNP Effects (Balanced Pleiotropy)",
      x = "SNP-exposure effect (gamma_hat)",
      y = "SNP-outcome effect (Gamma_hat)"
    ) +
    scale_color_manual(
      name = "Line Type",
      values = c("IVW Regression" = "orange", "True Effect" = "black")
    ) +
    theme_minimal() +
    theme(legend.position = "right",
          legend.text = element_text(size = 18),
          legend.title = element_text(size = 18, face = "bold"),
          axis.title = element_text(size = 18),
          plot.title = element_text(hjust = 0.5, face = "bold", size = 16) 
          )
}, width = 800, height = 600)